In [ ]:
import os
import h5py    
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
import cv2
import nrrd
from tifffile import tifffile
import json
from pprint import pprint
import math

In [ ]:
filepath='/net/birdstore/Vessel/WBIM/Acquisition/LifeCanvas/003_20240209/00005/Scan/00060_00089/00000/tile.h5'

In [ ]:
f = h5py.File(filepath, 'r')

In [ ]:
for key in f.keys():
    print(key) #Names of the root level object names in HDF5 file - can be groups or datasets.
    print(type(f[key])) # get the object type: usually group or dataset

In [ ]:
#Get the HDF5 group; key needs to be a group name from above
group = f['CH1']
print(type(group))
#Checkout what keys are inside that group.
for key in group.keys():
    print(key)

In [ ]:
data = group['raw'][()]

In [ ]:
print(f'dtype={data.dtype} shape={data.shape}')

In [ ]:
plt.rcParams["figure.figsize"] = [14, 10]
plt.rcParams["figure.autolayout"] = True
plt.imshow(data[125,:,:], cmap='seismic')
#plt.show()

In [ ]:
#filepath = '/home/eddyod/programming/ITKMontage/examples/SampleData_CMUrun2/tile_10.nrrd'
filepath = '/home/eddyod/programming/ITKMontage/examples/SampleData_DzZ_T1/tile_10.nrrd'
#filepath = '/home/eddyod/tmp/out.nrrd'
data, header = nrrd.read(filepath)
print(data.dtype, readdata.shape)

In [ ]:
filepath = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK20230126-003/preps/layers/00001/tif/00061_00100.tif'
data = tifffile.imread(filepath)
#data = np.swapaxes(data, 0, 2)
data.dtype, data.shape, data.ndim

In [ ]:
arr = data[0,:,:]
arr.dtype, arr.shape, arr.ndim

In [ ]:
#plt.rcParams["figure.figsize"] = [14, 10]
#plt.rcParams["figure.autolayout"] = True
#plt.imshow(data[12,:,:], cmap='gray')

In [ ]:
INPUT = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK20230126-003/preps/layers/00001'
infopath = os.path.join(INPUT, 'info')
infos = sorted(os.listdir(infopath))

In [ ]:
bbox_mmxx_um = []
bbox_mmxx_pxl = []
layer_z_um = []
stitch_tiles = []
for file in sorted(infos):
    inpath = os.path.join(infopath, file)
    with open(inpath) as json_data:
        d = json.load(json_data)
        stitch_tiles.append(d)
        json_data.close()
        #print(file, end="\t")
        #pprint(d['stack_size'])
        bbox_mmxx_um.append(d['tile_mmxx_um'])
        bbox_mmxx_pxl.append(d['tile_mmxx_pxl'])
        layer_z_um.append(d['layer_z_um'])

# set to np arrays
bbox_mmxx_um = np.array(bbox_mmxx_um)


In [ ]:
#tmp_stitch_data = zeros(ds_bbox_ll, 'uint16');
tmp_stitch_data = np.zeros(ds_bbox_ll[0:2], dtype=np.uint16)
tmp_stitch_data.dtype, tmp_stitch_data.shape

In [ ]:
tifpath = os.path.join(INPUT, 'tif')
tiles = sorted(os.listdir(tifpath))
# Parameters
#stitch_voxel_size_um = [2, 2, 2];
stitch_voxel_size_um = [1, 1, 1];
zero_num_sec = 0;
zero_last_section_Q = True
medfilt_Q = False
stack_size_um = stitch_tiles[0]['stack_size_um'];
print('stack_size_um', stack_size_um)
stack_size = stitch_tiles[0]['stack_size'];
#ds_stack_size = np.round(stack_size_um / stitch_voxel_size_um);
ds_stack_size = [round(stack/stitch) for stack,stitch in zip(stack_size_um, stitch_voxel_size_um)]
print('ds_stack_size', ds_stack_size)
#MATLAB bbox_mmxx_pxl = cat(1, stitch_tiles.tile_mmxx_pxl);
#MATLAB vol_bbox_z_mx_um = [min(layer_z_um), max(layer_z_um) + stack_size_um(3) - 1];
#MATLAB vol_bbox_mm_um = [min(bbox_mmxx_um(:, 1:2), [], 1), vol_bbox_z_mx_um(1)];
#MATLAB vol_bbox_xx_um = [max(bbox_mmxx_um(:, 3:4), [], 1), vol_bbox_z_mx_um(2)];
#MATLAB vol_bbox_ll_um = vol_bbox_xx_um - vol_bbox_mm_um + 1;


vol_bbox_z_mx_um = [56770, 56770, 249]
vol_bbox_mm_um = np.min(bbox_mmxx_um,0)[0:2].tolist()
vol_bbox_xx_um = np.max(bbox_mmxx_um,0)[2:4].tolist()
vol_bbox_mm_um.append(vol_bbox_z_mx_um[0])
vol_bbox_xx_um.append(vol_bbox_z_mx_um[1])
print('vol_bbox_mm_um', vol_bbox_mm_um)
print('vol_bbox_xx_um', vol_bbox_xx_um)

vol_bbox_ll_um = [a_i - b_i for a_i, b_i in zip(vol_bbox_xx_um, vol_bbox_mm_um)]
vol_bbox_ll_um = [a+1 for a in vol_bbox_ll_um]

print('vol_bbox_ll_um', vol_bbox_ll_um)
ds_bbox_ll = (np.array(vol_bbox_ll_um) / stitch_voxel_size_um)
ds_bbox_ll = [math.ceil(a) for a  in ds_bbox_ll]
#ds_bbox_ll = (np.array(vol_bbox_ll_um) / stitch_voxel_size_um)
print('ds_bbox_ll', ds_bbox_ll)
#print(f'vol_bbox_xx_um - vol_bbox_mm_um = {np.array(vol_bbox_xx_um) - np.array(vol_bbox_mm_um)}')

assert len(infos) == len(tiles), "Error, number of tiles does not equal number of json files"
for info,tile in zip(stitch_tiles, tiles):
    tmp_tile = info
    tmp_tile_data_path = os.path.join(tifpath, tile)
    tmp_tile_data = io.imread(tmp_tile_data_path)
    tmp_tile_data = np.swapaxes(tmp_tile_data, 0, 2)
    
    #tmp_tile_data = tmp_tile_data[i];
    #if zero_num_sec and (tmp_tile.layer > 1):
    #    tmp_tile_data[:, :, 1:zero_num_sec] = 0;
    
    #tmp_tile_data[:, :, -1] = 0;
    tmp_tile_bbox_mm_um = tmp_tile['tile_mmxx_um'][:2]
    tmp_tile_bbox_mm_um.append(tmp_tile['layer_z_um'])
    #tmp_tile_bbox_mm_um = [tmp_tile.tile_mmxx_um(1:2), tmp_tile.layer_z_um];    
    #tmp_tile_bbox_ll_um = [tmp_tile.tile_mmll_um(3:4), tmp_tile.stack_size_um(3)];
    tmp_tile_bbox_ll_um = tmp_tile['tile_mmll_um'][2:]
    tmp_tile_bbox_ll_um.append(tmp_tile['stack_size_um'][2])
    #tmp_tile_ll_ds_pxl = round(tmp_tile_bbox_ll_um ./ stitch_voxel_size_um);
    tmp_tile_ll_ds_pxl = [round(bbox/voxel) for bbox,voxel in zip(tmp_tile_bbox_ll_um, stitch_voxel_size_um)]
    print('TIF dtype and orignal shape',tile, tmp_tile_data.dtype, tmp_tile_data.shape, end="\t" )
    print('tmp_tile_ll_ds_pxl', tmp_tile_ll_ds_pxl)
    """
    # Downsample image stack - need smoothing? 
    tmp_tile_data = imresize3(tmp_tile_data, tmp_tile_ll_ds_pxl);
    tmp_tile.clear_buffer();
    # Local bounding box 
    tmp_local_bbox_um = tmp_tile_bbox_mm_um - vol_bbox_mm_um;
    tmp_local_bbox_mm_ds_pxl = round(tmp_local_bbox_um ./ stitch_voxel_size_um);
    # Deal with edge: 
    tmp_local_bbox_mm_ds_pxl = max(tmp_local_bbox_mm_ds_pxl, 1);
    tmp_local_bbox_xx_ds_pxl = tmp_local_bbox_mm_ds_pxl + tmp_tile_ll_ds_pxl - 1;
    # Max - rendering
    tmp_stitch_data(tmp_local_bbox_mm_ds_pxl(1) : tmp_local_bbox_xx_ds_pxl(1), ...
        tmp_local_bbox_mm_ds_pxl(2) : tmp_local_bbox_xx_ds_pxl(2), ...
        tmp_local_bbox_mm_ds_pxl(3) : tmp_local_bbox_xx_ds_pxl(3)) = max(tmp_stitch_data(...
        tmp_local_bbox_mm_ds_pxl(1) : tmp_local_bbox_xx_ds_pxl(1), ...
        tmp_local_bbox_mm_ds_pxl(2) : tmp_local_bbox_xx_ds_pxl(2), ...
        tmp_local_bbox_mm_ds_pxl(3) : tmp_local_bbox_xx_ds_pxl(3)), tmp_tile_data);
    """